# Conversion Tools
This a notebook containing various scripts to convert things

## 1 - convert a pytorch model to something I can use in keras

In [ ]:
import torch
import torchvision.models as models
import onnx
from onnx2keras import onnx_to_keras

In [ ]:
fer_model = models.resnet18(num_classes=7)

checkpoint = torch.load('../papers/Emotion-FAN/pretrain_model/Resnet18_FER+_pytorch.pth.tar', map_location=torch.device('cpu'))
pretrained_state_dict = checkpoint['state_dict']
model_state_dict = fer_model.state_dict()

for key in pretrained_state_dict:
    if ((key == 'module.fc.weight') | (key == 'module.fc.bias')):
        pass
    else:
        model_state_dict[key.replace('module.', '')] = pretrained_state_dict[key]

fer_model.load_state_dict(model_state_dict, strict=False)
model = torch.nn.DataParallel(fer_model)

model.eval()

x = torch.randn(64, 3, 244, 244, requires_grad=True)

torch.onnx.export(model.module, x, "../models/fan-fer.onnx", input_names= ['input'], 
                  dynamic_axes={'input':{0:'batch'}})



In [ ]:
onnx_model = onnx.load("../models/fan-fer.onnx")
onnx.checker.check_model(onnx_model)
k_model = onnx_to_keras(onnx_model, ['input'])
k_model.save('../models/fan-fer')

In [ ]:

ms1m_model = models.resnet18(num_classes=7)

checkpoint = torch.load('../papers/Emotion-FAN/pretrain_model/Resnet18_MS1M_pytorch.pth.tar', map_location=torch.device('cpu'))
pretrained_state_dict = checkpoint['state_dict']
model_state_dict = ms1m_model.state_dict()

for key in pretrained_state_dict:
    if ((key == 'module.fc.weight') | (key == 'module.fc.bias')):
        pass
    else:
        model_state_dict[key.replace('module.', '')] = pretrained_state_dict[key]

ms1m_model.load_state_dict(model_state_dict, strict=False)
model = torch.nn.DataParallel(ms1m_model)

model.eval()

x = torch.randn(64, 3, 244, 224, requires_grad=True)

# if using MS1M, then fix this which goes wrong for some reason
torch.onnx.export(model.module, x, "../models/fan-ms1m.onnx")


In [ ]:
onnx_model = onnx.load("../models/fan-ms1m.onnx")
onnx.checker.check_model(onnx_model)
k_model = onnx_to_keras(onnx_model, ['input.1'])
k_model.save('../models/fan-ms1m')

## Model from https://github.com/WuJie1010/Facial-Expression-Recognition.Pytorch
 Takes input of size 28x28